# Text Reorderer

This notebook provides a method to reorder the sentences in a text, using BERT For Next Sentence Prediction.
It is an important NLP task, especially when you have some unordered text coming from different sources such as in Summarization o Multi-Summarization tasks.  

## Required dependencies and set-up

The `transformers` library from HuggingFace to import BertTokenizer, BertForNextSentencePrediction is used. 
The tokenizer is used to tokenize the senteces used in the Next-Sentence-Prediction Model. The `bert-base-multilingual-cased` as pretrained weights are used to support cased text and several languages.

The `tensorflow` library is used to build the tensors for the model.
Finally, `tensorflow.keras` is used later to compute the softmax from the logits in the Next-Sentence-Prediction task.

In [ ]:
!pip install transformers nltk tensorflow

In [13]:
from transformers import BertTokenizer, TFBertForNextSentencePrediction
import tensorflow as tf
import tensorflow.keras

In [6]:
pretrained_weights = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
nsp_model = TFBertForNextSentencePrediction.from_pretrained(pretrained_weights)

## Functions

In this section, the function needed for the text reordering are explained.

### predict_next_sentence_prob
The `predict_next_sentence_prob` function aims to return the probability that the second sentence provided in input is the continuation of the first one.

In [18]:
def predict_next_sentence_prob(sent1, sent2):
    # encode the two sequences. Particularly, make clear that they must be 
    # encoded as "one" input to the model by using 'seq_B' as the 'text_pair'
    # NOTE how the token_type_ids are 0 for all tokens in seq_A and 1 for seq_B, 
    # this way the model knows which token belongs to which sequence
    encoded = tokenizer.encode_plus(sent1, text_pair=sent2)
    encoded["input_ids"] = tf.constant(encoded["input_ids"])[None, :]
    encoded["token_type_ids"] = tf.constant(encoded["token_type_ids"])[None, :]
    encoded["attention_mask"] = tf.constant(encoded["attention_mask"])[None, :]

    # a model's output is a tuple, we only need the output tensor containing
    # the relationships which is the first item in the tuple
    outputs = nsp_model(encoded)
    seq_relationship_scores = outputs[0]

    # we need softmax to convert the logits into probabilities
    # index 0: sequence B is a continuation of sequence A
    # index 1: sequence B is a random sequence
    probs = tf.keras.activations.softmax(seq_relationship_scores, axis=-1)
    return probs.numpy()[0][0]

In the following snippet the Next-Sentence-Prediction probabilities over two sentences are computed. You can see from the examples the results.

In [90]:
sentences = [
    "So, the old man decided to go home to make his dog feel better.",
    "In the park, there were also a mouse and the dog was frightened.",
    "The dog felt better because the man gave 3 biscuits to him.",
    "One day, an old man went to the park with his dog.",
]


print("Probability that sent2 is the next sentence with respect to sent1:",
      predict_next_sentence_prob(sentences[0], sentences[1]))
print("Probability that sent1 is the next sentence with respect to sent2:",
      predict_next_sentence_prob(sentences[1], sentences[0])) # better
print("Probability that a sentence is the next sentence with respect to itself:",
      predict_next_sentence_prob(sentences[1], sentences[1])) # very low

Probability that sent2 is the next sentence with respect to sent1: 0.9987509
Probability that sent1 is the next sentence with respect to sent2: 0.99957067
Probability that a sentence is the next sentence with respect to itself: 0.008009296


### create_correlation_matrix
The `create_correlation_matrix` takes a string array-like and create a correlation matrix which takes care of the all probabilities computed over the sentences given in input.

In [78]:
def create_correlation_matrix(sentences: list):
    num_sentences = len(sentences) 
    correlation_matrix = np.empty(shape=(num_sentences, num_sentences), dtype=float, order='C')
    for i, s1 in enumerate(sentences):
        for j, s2 in enumerate(sentences):
            correlation_matrix[i][j] = predict_next_sentence_prob(s1, s2)
    return correlation_matrix

In [74]:
correlation_matrix = create_correlation_matrix(sentences)
correlation_matrix

array([[0.00227171, 0.99875093, 0.99981552, 0.99655944],
       [0.99957067, 0.0080093 , 0.9995746 , 0.99964833],
       [0.99780148, 0.99955863, 0.00385047, 0.99557221],
       [0.99929106, 0.99986291, 0.99985898, 0.00212961]])

### reorder_sentences
The `reorder_sentences` takes a string array-like. It creates a correlation matrix with the previous function and iteratively finds the best association in the matrix. It deletes the sentences selected in this way and continues until no association can be done. The association is expressed as the tuple (X, Y), where X is the first sentence and the Y is the most probable next sentence. The latter is used as the new first sentence from which search the best next one. 

In [75]:
def reorder_sentences(sentences: list):
    ordering = []
    correlation_matrix = create_correlation_matrix(sentences)
    hint = None
    while correlation_matrix.any():
        if hint == None:
            ind = np.unravel_index(np.argmax(correlation_matrix, axis=None), correlation_matrix.shape)
        else:
            ind = np.unravel_index(np.argmax(correlation_matrix[hint,:], axis=None), correlation_matrix[hint,:].shape)
            ind = (hint, ind[0])
        hint = ind[1]    
        correlation_matrix[ind[0], :] = 0
        correlation_matrix[:, ind[0]] = 0
        ordering.append(ind[0])
    return ordering

In [91]:
ordering = reorder_sentences(sentences)
reordered_sentences = [sentences[idx] for idx in ordering]
ordering, reordered_sentences

([3, 1, 0, 2],
 ['One day, an old man went to the park with his dog.',
  'In the park, there were also a mouse and the dog was frightened.',
  'So, the old man decided to go home to make his dog feel better.',
  'The dog felt better because the man gave 3 biscuits to him.'])

## Example

This example shows how the text-reodered works over a medium long non-ordered sentences. Correct order is [2, 0, 1, 3, 5, 4], but it found another that fits well [2, 0, 1, 5, 4, 3]. Let's read the text to get a proof.


In [107]:
sentences =  [
    "famous thanks to the victories obtained during the first campaign of Italy, after the coup d'etat of the 18th brumaire (9 November 1799) he assumed power in france: he was first consul from November of that year to 18 May 1804, and emperor of the French, with the name of napoleon i (napoléon ier) from 2 December 1804 to 14 April 1814 and again from 20 March to 22 June 1815 .",
    "thanks to his system of alliances and a series of brilliant victories against European powers, he conquered and governed a large part of continental Europe, exporting the revolutionary ideals of social renewal and managing to control numerous kingdoms through people loyal to him (giuseppe bonaparte in spain , joachim murat in the kingdom of naples, girolamo bonaparte in westphalia, jean-baptiste jules bernadotte in the kingdom of sweden and luigi bonaparte in the kingdom of holland) .",
    "napoleon bonaparte (ajaccio, 15 august 1769 - longwood, sant'elena island, 5 may 1821) was a french politician and general, founder of the first french empire and protagonist of the first phase of european contemporary history called napoleonic age .",
    "great man of war, protagonist of over twenty years of campaigns in europe, napoleon was considered the greatest strategist in history by the military historian basil liddell hart, while the historian evgenij tàrle does not hesitate to define him `` the incomparable master of art of the war `` is `` the greatest of the great '' .",
    "in march 1815, stealthily abandoned the island, he landed in Golfe Juan, near Antibes and returned to Paris without encountering opposition, regaining power for the so-called << one hundred days >> period, until he was definitively defeated by the seventh coalition in the battle of Waterloo, 18 June 1815; he spent the last years of his life in exile on the island of Saint Helena, under the control of the British .",
    "napoleon was defeated in the battle of Leipzig by the European allies in October 1813 and he abdicated on April 4, 1814, and was exiled to the island of Elba : it marked the decline of his dominion over Europe .",
]
ordering = reorder_sentences(sentences)
reordered_sentences = [sentences[idx] for idx in ordering]
ordering, reordered_sentences

([2, 0, 1, 5, 4, 3],
 ["napoleon bonaparte (ajaccio, 15 august 1769 - longwood, sant'elena island, 5 may 1821) was a french politician and general, founder of the first french empire and protagonist of the first phase of european contemporary history called napoleonic age .",
  "famous thanks to the victories obtained during the first campaign of Italy, after the coup d'etat of the 18th brumaire (9 November 1799) he assumed power in france: he was first consul from November of that year to 18 May 1804, and emperor of the French, with the name of napoleon i (napoléon ier) from 2 December 1804 to 14 April 1814 and again from 20 March to 22 June 1815 .",
  'thanks to his system of alliances and a series of brilliant victories against European powers, he conquered and governed a large part of continental Europe, exporting the revolutionary ideals of social renewal and managing to control numerous kingdoms through people loyal to him (giuseppe bonaparte in spain , joachim murat in the kingd